In [16]:
# Cheng Dang 2021,10

# Check Mie Parameter

#  *** CRTM CODE ***:
#  --- CRTM_RTSolution.f90 ---
#  FUNCTION CRTM_Compute_nStreams( &
#     Atmosphere  , &  ! Input
#     SensorIndex , &  ! Input
#     ChannelIndex, &  ! Input
#     RTSolution  ) &  ! Output
#   RESULT( nStreams )

#     ! Determine the maximum cloud particle size
#     maxReff = ZERO
#     DO n = 1, Atmosphere%n_Clouds
#       Reff = MAXVAL(Atmosphere%Cloud(n)%Effective_Radius)
#       IF( Reff > maxReff) maxReff = Reff
#     END DO
#     DO n = 1, Atmosphere%n_Aerosols
#       Reff = MAXVAL(Atmosphere%Aerosol(n)%Effective_Radius)
#       IF( Reff > maxReff) maxReff = Reff
#     END DO
    
#     ! Compute the Mie parameter, 2.pi.Reff/lambda
#     MieParameter = TWO * PI * maxReff * SC(SensorIndex)%Wavenumber(ChannelIndex)/10000.0_fp

#     ! Determine the number of streams based on Mie parameter
#     IF ( MieParameter < 0.01_fp ) THEN
#       nStreams = 2
#     ELSE IF( MieParameter < ONE ) THEN
#       nStreams = 4
#     ELSE
#       nStreams = 6
#     END IF

#   --- Spectral_Units_Conversion.f90 ---  
#   REAL(fp), PARAMETER, PUBLIC :: SPEED_OF_LIGHT = 2.99792458e+08_fp
#   C => SPEED_OF_LIGHT
#
#   ELEMENTAL FUNCTION GHz_to_inverse_cm( Frequency ) RESULT( Wavenumber )
#     REAL(fp), INTENT(IN) :: Frequency
#     REAL(fp)             :: Wavenumber
#     REAL(fp), PARAMETER  :: SCALE_FACTOR = 1.0e+07_fp
#     IF ( Frequency < EPSILON(ONE) ) THEN
#       Wavenumber = ZERO
#       RETURN
#     END IF
#     Wavenumber = SCALE_FACTOR * Frequency / C
#   END FUNCTION GHz_to_inverse_cm

In [17]:
from netCDF4 import Dataset
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import math

In [29]:
def readnc(infile, varname):
    nc_fid = Dataset(infile,'r')
    out =  np.array(nc_fid.variables[varname][:])
    return out

def GHz_to_inverse_cm(frq_mtx):
    C = 2.99792458e8 #Speed of light
    SCALE_FACTOR = 1.0e7
    wvn_mtx = [(SCALE_FACTOR * i / C) for i in frq_mtx]
#     wvn_mtx = np.zeros(len(frq_mtx))    
#     for i, frq in enumerate(frq_mtx):
#         wvn_mtx[i] = SCALE_FACTOR * frq / C
    return wvn_mtx
  
def compute_nstr(Reff, wvn):
    # MieParameter = TWO * PI * maxReff * SC(SensorIndex)%Wavenumber(ChannelIndex)/10000.0_fp
    MP = 2.0 * math.pi * Reff * wvn / 1e4
    if MP < 0.01:
        NSTR = 2
    elif MP < 1:
        NSTR = 4
    else:
        NSTR = 6
    return NSTR

In [30]:
CloudLUT = '/Users/dangch/Documents/CRTM/CRTM_dev/crtm_code_review/fix/CloudCoeff/netCDF/CloudCoeff.nc4'
CloudCoeff = Dataset(CloudLUT)
CloudCoeff

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    write_module_history: $Id: CloudCoeff_netCDF_IO.f90 6760 2010-02-25 16:02:55Z paul.vandelst@noaa.gov $
    creation_date_and_time: 2010/02/25, 13:03:14 -0500UTC
    Release: 3
    Version: 4
    Title: CRTM Cloud Optical Properties in Infrared/Visible and Microwave Ranges
    History: $Id: CloudCoeff_R2_to_R3.f90 6765 2010-02-25 18:02:26Z paul.vandelst@noaa.gov $
    Comment: All MW, IR/VIS (note variable with _IR actual for both IR and VIS) liquid phase and solid phase with  the density < 0.9 g/cm3 are generated using a MIE code (Simmer, 1994). IR and visible solid phase with the density = 0.9 g/cm3 is adopted  from non-spherical particle of P. Yang (Liou and Yang, 1995) The asymmetry factor for non-spherical particles is used for the phase function.
    dimensions(sizes): n_MW_Frequencies(31), n_MW_Radii(10), n_IR_Frequencies(61), n_IR_Radii(10), n_Temperatures(5), n_Densities(3), n_IR_Densities

In [31]:
# Microwave
Reff_MW = CloudCoeff['Reff_MW'][:]
Frequency_MW = CloudCoeff['Frequency_MW'][:]
Wavenumber_MW = GHz_to_inverse_cm(Frequency_MW)
nstr_MW = np.zeros((len(Reff_MW), len(Wavenumber_MW)))
for i, Reff in enumerate(Reff_MW):
    for j, wvn in enumerate(Wavenumber_MW):
        nstr_MW[i,j] = compute_nstr(Reff, wvn)
        
print('nStream, Microwave (n_Reff_MW * n_Frequency_MW) :')
print(nstr_MW)

nStream, Microwave (n_Reff_MW * n_Frequency_MW) :
[[2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
  2. 2. 2. 4. 4. 4. 4.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.
  4. 4. 4. 4. 4. 4. 4.]
 [2. 2. 2. 2. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.
  4. 4. 4. 4. 4. 4. 4.]
 [2. 2. 2. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.
  4. 4. 4. 4. 4. 4. 4.]
 [2. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.
  4. 4. 4. 4. 4. 4. 4.]
 [2. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.
  4. 4. 4. 4. 4. 6. 6.]
 [4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.
  4. 4. 4. 6. 6. 6. 6.]
 [4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 6. 6. 6. 6.
  6. 6. 6. 6. 6. 6. 6.]
 [4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6.
  6. 6. 6. 6. 6. 6. 6.]
 [4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 

In [32]:
# IR
Reff_IR = CloudCoeff['Reff_IR'][:]
Frequency_IR = CloudCoeff['Frequency_IR'][:]
Wavenumber_IR = GHz_to_inverse_cm(Frequency_IR)
nstr_IR = np.zeros((len(Reff_IR), len(Wavenumber_IR)))
for i, Reff in enumerate(Reff_IR):
    for j, wvn in enumerate(Wavenumber_IR):
        nstr_IR[i,j] = compute_nstr(Reff, wvn)
        
print('nStream, IR (n_Reff_IR * n_Frequency_IR) :')
print(nstr_IR)

nStream, IR (n_Reff_IR * n_Frequency_IR) :
[[2. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.
  4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.
  4. 4. 4. 4. 4. 4. 4. 4. 4. 6. 6. 6. 6.]
 [4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.
  4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.
  4. 4. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6.]
 [4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.
  4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 6. 6. 6. 6.
  6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6.]
 [4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.
  4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 6. 6. 6. 6. 6. 6. 6. 6. 6.
  6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6.]
 [4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.
  4. 4. 4. 4. 4. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6.
  6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6.]
 [4. 4.